## Advanced Housing Prices - Feature Engineering

### we will be performing the following steps in this Feature Engineering Section
1. Missing Values
2. Temporal Variables
3. Categorical Variables: Remove rare Labels
4. Standardise the values of the variables to the same range

In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.pandas.set_option('display.max_columns', None)

In [34]:
dataset = pd.read_csv("datasets/house-prices-advanced-regression-techniques/train.csv")
# data =  dataset.copy()
# dataset = dataset.drop(columns=['Id'])
dataset.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [35]:
##Always rememeber that there will always be data leakageso we need to split the data first and then apply feature engineering
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(dataset.drop(columns=['SalePrice']), dataset['SalePrice'], test_size=0.2, random_state=42)
# X_train.shape, X_test.shape, y_train.shape, y_test.shape

## Dealing with Missing Values in Categorical, then Numerical Features

In [36]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [37]:
categorical_features_with_nan = [feature for feature in dataset.columns if dataset[feature].isnull().sum()>1 and dataset[feature].dtype == 'O']

for feature in categorical_features_with_nan:
    print("{}:{} % missing values".format(feature,np.round(dataset[feature].isnull().sum()/len(dataset[feature])*100)))

Alley:94.0 % missing values
MasVnrType:60.0 % missing values
BsmtQual:3.0 % missing values
BsmtCond:3.0 % missing values
BsmtExposure:3.0 % missing values
BsmtFinType1:3.0 % missing values
BsmtFinType2:3.0 % missing values
FireplaceQu:47.0 % missing values
GarageType:6.0 % missing values
GarageFinish:6.0 % missing values
GarageQual:6.0 % missing values
GarageCond:6.0 % missing values
PoolQC:100.0 % missing values
Fence:81.0 % missing values
MiscFeature:96.0 % missing values


In [38]:
#For categorical features with missing values, we can replace them with the string "missing" to indicate that they are missing which is a category in itself and is a common prractice
def replace_cat_feature(dataset,categorical_features_with_nan):
    data = dataset.copy()
    # for feature in categorical_features_with_nan:
    data[categorical_features_with_nan] = data[categorical_features_with_nan].fillna("Missing")
    return data
    
dataset = replace_cat_feature(dataset,categorical_features_with_nan)

dataset[categorical_features_with_nan].isnull().sum() #Check if there are any missing values


Alley           0
MasVnrType      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
FireplaceQu     0
GarageType      0
GarageFinish    0
GarageQual      0
GarageCond      0
PoolQC          0
Fence           0
MiscFeature     0
dtype: int64

In [39]:
#Now we can check the categorical features with missing values
# dataset.head(100)
categorical_features_with_nan
for feature in categorical_features_with_nan:
    print("{}:{} % missing values".format(feature,np.round(dataset[feature].isnull().sum()/len(dataset[feature])*100)))

Alley:0.0 % missing values
MasVnrType:0.0 % missing values
BsmtQual:0.0 % missing values
BsmtCond:0.0 % missing values
BsmtExposure:0.0 % missing values
BsmtFinType1:0.0 % missing values
BsmtFinType2:0.0 % missing values
FireplaceQu:0.0 % missing values
GarageType:0.0 % missing values
GarageFinish:0.0 % missing values
GarageQual:0.0 % missing values
GarageCond:0.0 % missing values
PoolQC:0.0 % missing values
Fence:0.0 % missing values
MiscFeature:0.0 % missing values


### Clearly all the missing values were replaced with the string "Missing"

In [40]:
dataset.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Missing,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,Missing,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,Missing,Missing,Missing,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Missing,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,Missing,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,Missing,Missing,Missing,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,Missing,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,Missing,Missing,Missing,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,Missing,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,Missing,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,Missing,Missing,Missing,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,Missing,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,Missing,Missing,Missing,0,12,2008,WD,Normal,250000


In [41]:
#Now we fill the missing values for numerical feature with median values in the feature to avoid the effect of outliers in the data/feature/variable
numerical_features_with_nan = [feature for feature in dataset.columns if dataset[feature].isnull().sum() > 1 and dataset[feature].dtype != 'O' ]
# numerical_features_with_nan
for feature in numerical_features_with_nan:
    print("{}:{} % missing values".format(feature, np.round(dataset[feature].isnull().sum()/len(dataset[feature])*100)))

LotFrontage:18.0 % missing values
MasVnrArea:1.0 % missing values
GarageYrBlt:6.0 % missing values


In [42]:
## Here we will replace the missing values with the median value of the feature
## Also we will create a new feature namely feature_name+'_NaN' to indicate the original feature values which were missing and which were not

for feature in numerical_features_with_nan:
    median_value = dataset[feature].median()

    dataset[feature+'_NaN'] = np.where(dataset[feature].isnull(),1,0)
    dataset[feature] = dataset[feature].fillna(median_value)

dataset[numerical_features_with_nan].isnull().sum()


LotFrontage    0
MasVnrArea     0
GarageYrBlt    0
dtype: int64

In [43]:
dataset.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,LotFrontage_NaN,MasVnrArea_NaN,GarageYrBlt_NaN
0,1,60,RL,65.0,8450,Pave,Missing,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,Missing,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,Missing,Missing,Missing,0,2,2008,WD,Normal,208500,0,0,0
1,2,20,RL,80.0,9600,Pave,Missing,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,Missing,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,Missing,Missing,Missing,0,5,2007,WD,Normal,181500,0,0,0
2,3,60,RL,68.0,11250,Pave,Missing,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,Missing,Missing,Missing,0,9,2008,WD,Normal,223500,0,0,0
3,4,70,RL,60.0,9550,Pave,Missing,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,Missing,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,Missing,Missing,Missing,0,2,2006,WD,Abnorml,140000,0,0,0
4,5,60,RL,84.0,14260,Pave,Missing,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,Missing,Missing,Missing,0,12,2008,WD,Normal,250000,0,0,0
5,6,50,RL,85.0,14115,Pave,Missing,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,Missing,0.0,TA,TA,Wood,Gd,TA,No,GLQ,732,Unf,0,64,796,GasA,Ex,Y,SBrkr,796,566,0,1362,1,0,1,1,1,1,TA,5,Typ,0,Missing,Attchd,1993.0,Unf,2,480,TA,TA,Y,40,30,0,320,0,0,Missing,MnPrv,Shed,700,10,2009,WD,Normal,143000,0,0,0
6,7,20,RL,75.0,10084,Pave,Missing,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186.0,Gd,TA,PConc,Ex,TA,Av,GLQ,1369,Unf,0,317,1686,GasA,Ex,Y,SBrkr,1694,0,0,1694,1,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2004.0,RFn,2,636,TA,TA,Y,255,57,0,0,0,0,Missing,Missing,Missing,0,8,2007,WD,Normal,307000,0,0,0
7,8,60,RL,69.0,10382,Pave,Missing,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,7,6,1973,1973,Gable,CompShg,HdBoard,HdBoard,Stone,240.0,TA,TA,CBlock,Gd,TA,Mn,ALQ,859,BLQ,32,216,1107,GasA,Ex,Y,SBrkr,1107,983,0,2090,1,0,2,1,3,1,TA,7,Typ,2,TA,Attchd,1973.0,RFn,2,484,TA,TA,Y,235,204,228,0,0,0,Missing,Missing,Shed,350,11,2009,WD,Normal,200000,1,0,0
8,9,50,RM,51.0,6120,Pave,Missing,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,1.5Fin,7,5,1931,1950,Gable,CompShg,BrkFace,Wd Shng,Missing,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,952,952,GasA,Gd,Y,FuseF,1022,752,0,1774,0,0,2,0,2,2,TA,8,Min1,2,TA,Detchd,1931.0,Unf,2,468,Fa,TA,Y,90,0,205,0,

In [44]:
# Handling temporal variables 
year_features = [feature for feature in dataset.columns if 'Yr' in feature or 'Year' in feature]
# year_features.remove('YrSold')
year_features
dataset[year_features].head(10)
# dataset['YrSold'].head(10)

,YearBuilt,YearRemodAdd,GarageYrBlt,YrSold,GarageYrBlt_NaN
0,2003,2003,2003.0,2008,0
1,1976,1976,1976.0,2007,0
2,2001,2002,2001.0,2008,0
3,1915,1970,1998.0,2006,0
4,2000,2000,2000.0,2008,0
5,1993,1995,1993.0,2009,0
6,2004,2005,2004.0,2007,0
7,1973,1973,1973.0,2009,0
8,1931,1950,1931.0,2008,0
9,1939,1950,1939.0,2008,0


In [45]:
if "YrSold" in dataset.columns:
    for feature in year_features:
        if feature != "YrSold":  # Skip 'YrSold' itself
            dataset[feature] = dataset["YrSold"] - dataset[feature]
else:
    print("Error: 'YrSold' column not found in dataset!")

In [46]:
dataset[year_features].head(10)

,YearBuilt,YearRemodAdd,GarageYrBlt,YrSold,GarageYrBlt_NaN
0,5,5,5.0,2008,2008
1,31,31,31.0,2007,2007
2,7,6,7.0,2008,2008
3,91,36,8.0,2006,2006
4,8,8,8.0,2008,2008
5,16,14,16.0,2009,2009
6,3,2,3.0,2007,2007
7,36,36,36.0,2009,2009
8,77,58,77.0,2008,2008
9,69,58,69.0,2008,2008


In [47]:
#Lets check if there are any missing values in the dataset after this point
dataset.isnull().sum() #Check if there are any missing values in the dataset

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage        0
LotArea            0
                  ..
SaleCondition      0
SalePrice          0
LotFrontage_NaN    0
MasVnrArea_NaN     0
GarageYrBlt_NaN    0
Length: 84, dtype: int64

### In eda file we saw continous variables were skewed and before applying log transformation we skipped those variables that had nan values

In [48]:
#numerical_features 
numerical_features  = [feature for feature in dataset.columns if dataset[feature].dtype != 'O' and feature not in ['SalePrice']]
# numerical_features
discrete_feature = [feature for feature in numerical_features if len(dataset[feature].unique())<25 and feature not in year_features+['Id']]
continuous_feature = [feature for feature in numerical_features if feature not in discrete_feature+year_features+['Id']]
print("Continuous feature Count {}".format(len(continuous_feature)))


Continuous feature Count 15


In [49]:
continuous_feature #Check if there are any missing values in the dataset    

['LotFrontage',
 'LotArea',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'GrLivArea',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 'ScreenPorch']

In [50]:
dataset[continuous_feature].head(10)

,LotFrontage,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,ScreenPorch
0,65.0,8450,196.0,706,0,150,856,856,854,1710,548,0,61,0,0
1,80.0,9600,0.0,978,0,284,1262,1262,0,1262,460,298,0,0,0
2,68.0,11250,162.0,486,0,434,920,920,866,1786,608,0,42,0,0
3,60.0,9550,0.0,216,0,540,756,961,756,1717,642,0,35,272,0
4,84.0,14260,350.0,655,0,490,1145,1145,1053,2198,836,192,84,0,0
5,85.0,14115,0.0,732,0,64,796,796,566,1362,480,40,30,0,0
6,75.0,10084,186.0,1369,0,317,1686,1694,0,1694,636,255,57,0,0
7,69.0,10382,240.0,859,32,216,1107,1107,983,2090,484,235,204,228,0
8,51.0,6120,0.0,0,0,952,952,1022,752,1774,468,90,0,205,0
9,50.0,7420,0.0,851,0,140,991,1077,0,1077,205,0,4,0,0


In [51]:
for feature in continuous_feature:
    if 0 in dataset[feature].unique():
        dataset[feature] = dataset[feature] + 1
        #We can add 1 to the feature to avoid log(0) which is undefined
    dataset[feature] = np.log(dataset[feature])
    # plt.figure(figsize=(12,10))
    # plt.hist(dataset[feature], bins=70, color='blue', alpha=0.7)
    # plt.xlabel(feature)
    # plt.ylabel('Count')
    # plt.title(feature)
    # plt.show()
#We can see that the distribution of the features is now normal and we can use these features for our model

### Above i have plotted the log transofrmed didtributions of all the continous variables. But not all the continous varibales need log trransform some need Box-Cox Transform or some other transform to remove the artifacts like skewness, outliers, high variance etc. Refer word file Feature Engineering.docx for details.  

#### Below i have used only those continous features which had non zero entries only and applied log transform and then I am gonna train the model/on these log transformed and other important feature that we select which we will see in the feature selection section.

In [52]:
continuous_feature_with_log = ['LotFrontage', 'LotArea', '1stFlrSF', 'GrLivArea','SalePrice']
for feature in continuous_feature_with_log:
    dataset[feature] = np.log(dataset[feature])

    

In [53]:
dataset[continuous_feature_with_log].head(10)

,LotFrontage,LotArea,1stFlrSF,GrLivArea,SalePrice
0,1.428968,2.201872,1.909879,2.007442,12.247694
1,1.477511,2.215885,1.965776,1.965776,12.109011
2,1.439718,2.233034,1.920501,2.013266,12.317167
3,1.409607,2.215315,1.926869,2.007990,11.849398
4,1.488584,2.258133,1.952057,2.040610,12.429216
5,1.491251,2.257064,1.899058,1.976399,11.870600
6,1.462674,2.221235,2.006178,2.006178,12.634603
7,1.443172,2.224389,1.947253,2.034041,12.206073
8,1.369104,2.165541,1.935790,2.012365,11.774520
9,1.364055,2.187391,1.943326,1.943326,11.678440


### Handling Rare Categorical Feature :- When a categorical feature has one category that appears in less than 1% of observations we relabel it as rare_val and then Validate Whether "Rare_Val" Impacts SalePrice. 
### This can be done by plotting boxplot agasint the sales price and checking the imapct on salesprice or use can also check statistical significance using ANNOVA

## If "Rare_Val" strongly influences SalePrice, it might signal the importance of less common but high-value properties. how do i find this give me code and explain. refer word file for detail. not everything might be correct.


# OR

### We can remove the rare_val category if it is not contributing to  and then give numerical values to the remaining category values 

##### There are many reason why rare_val vategories can affect the models prediction
1. Rare categories may not have enough data to generalize well, causing the model to learn noisy patterns instead of meaningful relationships.
2. - Rare categories might be underrepresented in training data and lead to biased predictions.



In [54]:
categorical_features=[feature for feature in dataset.columns if dataset[feature].dtype=='O']
dataset.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,LotFrontage_NaN,MasVnrArea_NaN,GarageYrBlt_NaN
0,1,60,RL,1.428968,2.201872,Pave,Missing,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,5,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,5.283204,Gd,TA,PConc,Gd,TA,No,GLQ,6.561031,Unf,0.000000,5.017280,6.753438,GasA,Ex,Y,SBrkr,1.909879,6.751101,0,2.007442,1,0,2,1,3,1,Gd,8,Typ,0,Missing,Attchd,5.0,RFn,2,6.308098,TA,TA,Y,0.000000,4.127134,0.000000,0,0.0,0,Missing,Missing,Missing,0,2,2008,WD,Normal,12.247694,0,0,2008
1,2,20,RL,1.477511,2.215885,Pave,Missing,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,31,31,Gable,CompShg,MetalSd,MetalSd,Missing,0.000000,TA,TA,CBlock,Gd,TA,Gd,ALQ,6.886532,Unf,0.000000,5.652489,7.141245,GasA,Ex,Y,SBrkr,1.965776,0.000000,0,1.965776,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,31.0,RFn,2,6.133398,TA,TA,Y,5.700444,0.000000,0.000000,0,0.0,0,Missing,Missing,Missing,0,5,2007,WD,Normal,12.109011,0,0,2007
2,3,60,RL,1.439718,2.233034,Pave,Missing,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,7,6,Gable,CompShg,VinylSd,VinylSd,BrkFace,5.093750,Gd,TA,PConc,Gd,TA,Mn,GLQ,6.188264,Unf,0.000000,6.075346,6.825460,GasA,Ex,Y,SBrkr,1.920501,6.765039,0,2.013266,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,7.0,RFn,2,6.411818,TA,TA,Y,0.000000,3.761200,0.000000,0,0.0,0,Missing,Missing,Missing,0,9,2008,WD,Normal,12.317167,0,0,2008
3,4,70,RL,1.409607,2.215315,Pave,Missing,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,91,36,Gable,CompShg,Wd Sdng,Wd Shng,Missing,0.000000,TA,TA,BrkTil,TA,Gd,No,ALQ,5.379897,Unf,0.000000,6.293419,6.629363,GasA,Gd,Y,SBrkr,1.926869,6.629363,0,2.007990,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,8.0,Unf,3,6.466145,TA,TA,Y,0.000000,3.583519,5.609472,0,0.0,0,Missing,Missing,Missing,0,2,2006,WD,Abnorml,11.849398,0,0,2006
4,5,60,RL,1.488584,2.258133,Pave,Missing,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,8,8,Gable,CompShg,VinylSd,VinylSd,BrkFace,5.860786,Gd,TA,PConc,Gd,TA,Av,GLQ,6.486161,Unf,0.000000,6.196444,7.044033,GasA,Ex,Y,SBrkr,1.952057,6.960348,0,2.040610,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,8.0,RFn,3,6.729824,TA,TA,Y,5.262690,4.442651,0.000000,0,0.0,0,Missing,Missing,Missing,0,12,2008,WD,Normal,12.429216,0,0,2008
5,6,50,RL,1.491251,2.257064,Pave,Missing,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,16,14,Gable,CompShg,VinylSd,VinylSd,Missing,0.000000,TA,TA,Wood,Gd,TA,No,GLQ,6.597146,Unf,0.000000,4.174387,6.680855,GasA,Ex,Y,SBrkr,1.899058,6.340359,0,1.976399,1,0,1,1,1,1,TA,5,Typ,0,Missing,Attchd,16.0,Unf,2,6.175867,TA,TA,Y,3.713572,3.433987,0.000000,320,0.0,0,Missing,MnPrv,Shed,700,10,2009,WD,Normal,11.870600,0,0,2009
6,7,20,RL,1.462674,2.221235,Pave,Missing,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,3,2,Gable,CompShg,VinylSd,VinylSd,Stone,5.231109,Gd,TA,PConc,Ex,TA,Av,GLQ,7.222566,Unf,0.000000,5.762051,7.430707,GasA,Ex,Y,SBrkr,2.006178,0.000000,0,2.006178,1,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,3.0,RFn,2,6.456770,TA,TA,Y,5.545177,4.060443,0.000000,0,0.0,0,Missing,Missing,Missing,0,8,2007,WD,Normal,12.634603,0,0,2007
7,8,60,RL,1.443172,2.224389,Pave,Missing,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,7,6,36,36,Gable,CompShg,HdBoard,HdBoard,Stone,5.484797

In [55]:
for feature in categorical_features:
    temp=dataset.groupby(feature)['SalePrice'].count()/len(dataset)
    temp_df=temp[temp>0.01].index
    dataset[feature]=np.where(dataset[feature].isin(temp_df),dataset[feature],'Rare_var')
    
    

In [56]:
dataset.head(100)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,LotFrontage_NaN,MasVnrArea_NaN,GarageYrBlt_NaN
0,1,60,RL,1.428968,2.201872,Pave,Missing,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,5,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,5.283204,Gd,TA,PConc,Gd,TA,No,GLQ,6.561031,Unf,0.0,5.017280,6.753438,GasA,Ex,Y,SBrkr,1.909879,6.751101,0,2.007442,1,0,2,1,3,1,Gd,8,Typ,0,Missing,Attchd,5.0,RFn,2,6.308098,TA,TA,Y,0.000000,4.127134,0.000000,0,0.0,0,Missing,Missing,Missing,0,2,2008,WD,Normal,12.247694,0,0,2008
1,2,20,RL,1.477511,2.215885,Pave,Missing,Reg,Lvl,AllPub,FR2,Gtl,Rare_var,Feedr,Norm,1Fam,1Story,6,8,31,31,Gable,CompShg,MetalSd,MetalSd,Missing,0.000000,TA,TA,CBlock,Gd,TA,Gd,ALQ,6.886532,Unf,0.0,5.652489,7.141245,GasA,Ex,Y,SBrkr,1.965776,0.000000,0,1.965776,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,31.0,RFn,2,6.133398,TA,TA,Y,5.700444,0.000000,0.000000,0,0.0,0,Missing,Missing,Missing,0,5,2007,WD,Normal,12.109011,0,0,2007
2,3,60,RL,1.439718,2.233034,Pave,Missing,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,7,6,Gable,CompShg,VinylSd,VinylSd,BrkFace,5.093750,Gd,TA,PConc,Gd,TA,Mn,GLQ,6.188264,Unf,0.0,6.075346,6.825460,GasA,Ex,Y,SBrkr,1.920501,6.765039,0,2.013266,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,7.0,RFn,2,6.411818,TA,TA,Y,0.000000,3.761200,0.000000,0,0.0,0,Missing,Missing,Missing,0,9,2008,WD,Normal,12.317167,0,0,2008
3,4,70,RL,1.409607,2.215315,Pave,Missing,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,91,36,Gable,CompShg,Wd Sdng,Wd Shng,Missing,0.000000,TA,TA,BrkTil,TA,Gd,No,ALQ,5.379897,Unf,0.0,6.293419,6.629363,GasA,Gd,Y,SBrkr,1.926869,6.629363,0,2.007990,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,8.0,Unf,3,6.466145,TA,TA,Y,0.000000,3.583519,5.609472,0,0.0,0,Missing,Missing,Missing,0,2,2006,WD,Abnorml,11.849398,0,0,2006
4,5,60,RL,1.488584,2.258133,Pave,Missing,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,8,8,Gable,CompShg,VinylSd,VinylSd,BrkFace,5.860786,Gd,TA,PConc,Gd,TA,Av,GLQ,6.486161,Unf,0.0,6.196444,7.044033,GasA,Ex,Y,SBrkr,1.952057,6.960348,0,2.040610,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,8.0,RFn,3,6.729824,TA,TA,Y,5.262690,4.442651,0.000000,0,0.0,0,Missing,Missing,Missing,0,12,2008,WD,Normal,12.429216,0,0,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,60,RL,1.443172,2.217742,Pave,Missing,IR2,Lvl,AllPub,Corner,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,8,16,16,Gable,CompShg,VinylSd,VinylSd,BrkFace,4.234107,Ex,Gd,PConc,Gd,Gd,No,ALQ,5.739793,Unf,0.0,5.916202,6.523562,GasA,Gd,Y,SBrkr,1.875195,6.673298,0,1.986917,0,0,2,1,3,1,TA,6,Typ,1,TA,BuiltIn,16.0,Fin,2,6.042633,TA,TA,Y,5.451038,4.158883,0.000000,0,0.0,0,Missing,Missing,Shed,480,4,2009,WD,Normal,12.128111,1,0,2009
96,97,20,RL,1.471717,2.223152,Pave,Missing,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,1Story,7,5,7,7,Gable,CompShg,VinylSd,VinylSd,BrkFace,5.214936,Gd,TA,PConc,Gd,TA,Av,ALQ,7.058758,Unf,0.0,6.056784,7.370860,GasA,Ex,Y,SBrkr,1.997449,0.000000,0,1.997449

In [57]:
# for feature in categorical_features:
#     labels_ordered=dataset.groupby([feature])['SalePrice'].mean().sort_values().index
#     labels_ordered={k:i for i,k in enumerate(labels_ordered,0)}
#     dataset[feature]=dataset[feature].map(labels_ordered)

# for feature in categorical_features:
#     # Step 1: Compute mean SalePrice for each category
#     labels_ordered = dataset.groupby([feature])['SalePrice'].mean().sort_values().index
    
#     # Step 2: Create target encoding dictionary
#     labels_ordered = {k: i for i, k in enumerate(labels_ordered, 0)}  # Assign rank-based numeric values
    
#     # Step 3: Assign 101 explicitly to "Rare_var"
#     labels_ordered["Rare_var"] = 101  
    
#     # Step 4: Apply encoding to the dataset
#     dataset[feature] = dataset[feature].map(labels_ordered)


for feature in categorical_features:
    # Step 1: Compute frequency of each category
    category_counts = dataset[feature].value_counts()  # Get count of each category
    
    # Step 2: Sort categories by frequency in ascending order (least common first)
    categories_sorted = category_counts.sort_values().index  
    
    # Step 3: Create ranking dictionary (lowest frequency = rank 0, highest = max rank)
    labels_ordered = {cat: i for i, cat in enumerate(categories_sorted)}
    
    # Step 4: Assign 101 explicitly to "Rare_var"
    # labels_ordered["Rare_var"] = -1  
    
    # Step 5: Apply encoding to the dataset
    dataset[feature] = dataset[feature].map(labels_ordered)

In [58]:
dataset.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,LotFrontage_NaN,MasVnrArea_NaN,GarageYrBlt_NaN
0,1,60,4,1.428968,2.201872,1,2,3,3,1,4,2,21,5,1,4,4,7,5,5,5,2,1,10,10,2,5.283204,2,3,4,3,4,4,5,6.561031,6,0.000000,5.017280,6.753438,2,4,1,3,1.909879,6.751101,0,2.007442,1,0,2,1,3,1,2,8,4,0,5,5,5.0,2,2,6.308098,3,3,2,0.000000,4.127134,0.000000,0,0.0,0,1,4,2,0,2,2008,3,4,12.247694,0,0,2008
1,2,20,4,1.477511,2.215885,1,2,3,3,1,1,2,3,4,1,4,5,6,8,31,31,2,1,8,9,3,0.000000,3,3,3,3,4,2,4,6.886532,6,0.000000,5.652489,7.141245,2,4,1,3,1.965776,0.000000,0,1.965776,0,1,2,0,3,1,3,6,4,1,3,5,31.0,2,2,6.133398,3,3,2,5.700444,0.000000,0.000000,0,0.0,0,1,4,2,0,5,2007,3,4,12.109011,0,0,2007
2,3,60,4,1.439718,2.233034,1,2,2,3,1,4,2,21,5,1,4,4,7,5,7,6,2,1,10,10,2,5.093750,2,3,4,3,4,1,5,6.188264,6,0.000000,6.075346,6.825460,2,4,1,3,1.920501,6.765039,0,2.013266,1,0,2,1,3,1,2,6,4,1,3,5,7.0,2,2,6.411818,3,3,2,0.000000,3.761200,0.000000,0,0.0,0,1,4,2,0,9,2008,3,4,12.317167,0,0,2008
3,4,70,4,1.409607,2.215315,1,2,2,3,1,3,2,11,5,1,4,4,7,5,91,36,2,1,7,4,3,0.000000,3,3,2,4,3,4,4,5.379897,6,0.000000,6.293419,6.629363,2,2,1,3,1.926869,6.629363,0,2.007990,1,0,1,0,3,1,2,7,4,1,4,4,8.0,3,3,6.466145,3,3,2,0.000000,3.583519,5.609472,0,0.0,0,1,4,2,0,2,2006,3,2,11.849398,0,0,2006
4,5,60,4,1.488584,2.258133,1,2,2,3,1,1,2,9,5,1,4,4,8,5,8,8,2,1,10,10,2,5.860786,2,3,4,3,4,3,5,6.486161,6,0.000000,6.196444,7.044033,2,4,1,3,1.952057,6.960348,0,2.040610,1,0,2,1,4,1,2,9,4,1,3,5,8.0,2,3,6.729824,3,3,2,5.262690,4.442651,0.000000,0,0.0,0,1,4,2,0,12,2008,3,4,12.429216,0,0,2008
5,6,50,4,1.491251,2.257064,1,2,2,3,1,4,2,10,5,1,4,3,5,5,16,14,2,1,10,10,3,0.000000,3,3,0,3,4,4,5,6.597146,6,0.000000,4.174387,6.680855,2,4,1,3,1.899058,6.340359,0,1.976399,1,0,1,1,1,1,3,5,4,0,5,5,16.0,3,2,6.175867,3,3,2,3.713572,3.433987,0.000000,320,0.0,0,1,3,1,700,10,2009,3,4,11.870600,0,0,2009
6,7,20,4,1.462674,2.221235,1,2,3,3,1,4,2,18,5,1,4,5,8,5,3,2,2,1,10,10,1,5.231109,2,3,4,2,4,3,5,7.222566,6,0.000000,5.762051,7.430707,2,4,1,3,2.006178,0.000000,0,2.006178,1,0,2,0,3,1,2,7,4,1,4,5,3.0,2,2,6.456770,3,3,2,5.545177,4.060443,0.000000,0,0.0,0,1,4,2,0,8,2007,3,4,12.634603,0,0,2007
7,8,60,4,1.443172,2.224389,1,2,2,3,1,3,2,14,0,1,4,4,7,6,36,36,2,1,9,8,1,5.484797,3,3,3,3,4,1,4,6.756932,2,3.496508,5.379897,7.010312,2,4,1,3,1.947253,6.891626,0,2.034041,1,0,2,1,3,1,3,7,4,2,3,5,36.0,2,2,6.184149,3,3,2,5.463832,5.323010,5.433722,0,0.0,0,1,4,1,350,11,2009,3,4,12.206073,1,0,2009
8,9,50,3,1.369104,2.165541,1,2,3,3,1,4,2,20,3,1,4,3,7,5,77,58,2,1,4,4,3,0.000000,3,3,2,4,4,4,6,0.000000,6,0.000000,6.859615,6.859615,2,2,1,1,1.935790,6.624065,0,2.012365,0,0,2,0,2,2,3,8,2,2,3,4,77.0,3,2,6.150603,1,3,2,4.510860,0.000000,5.327876,0,0.0,0,1,4,2,0,4,2008,3,2,11.774520,0,0,2008
9,10,190,4,1.364055,2.187391,1,2,3,3,1,3,2,12,3,0,0,0,5,6,69,58,2,1,8,9,3,0.000000,3,3,2,4,4,4,5,6.747587,6,0.000000,4.948760,6.899723,2,4,1,3,1.943326,0.000000,0,1.943326,1,0,1,0,2,2,3,5,4,2,3,5,69.0,2,1,5.327876,0,3,2,0.000000,1.609438,0.000000,0,0.0,0,1,4,2,0,1,2008,3,4,11.678440,0,0,2008


## Below code jsut plots the boxplots for the rare_var category in the categorical variables 

In [59]:
# import seaborn as sns
# for feature in categorical_features:
#     print("Unique Categories for {}:{}".format(feature, dataset[feature].unique()))
#     plt.figure(figsize=(10, 6))
#     sns.boxplot(x=dataset[feature], y=dataset["SalePrice"])
#     plt.xticks(rotation=45)
#     plt.title("SalePrice Distribution Across Categories")
#     plt.show()

### Feature Scaling


In [60]:
feature_scale=[feature for feature in dataset.columns if feature not in ['Id','SalePrice']]

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(dataset[feature_scale])

MinMaxScaler()

In [61]:
scaler.transform(dataset[feature_scale])

array([[0.23529412, 1.        , 0.49689213, ..., 0.        , 0.        ,
        0.6       ],
       [0.        , 1.        , 0.57331546, ..., 0.        , 0.        ,
        0.4       ],
       [0.23529412, 1.        , 0.51381745, ..., 0.        , 0.        ,
        0.6       ],
       ...,
       [0.29411765, 1.        , 0.50263957, ..., 0.        , 0.        ,
        1.        ],
       [0.        , 1.        , 0.51381745, ..., 0.        , 0.        ,
        1.        ],
       [0.        , 1.        , 0.54995643, ..., 0.        , 0.        ,
        0.6       ]], shape=(1460, 82))

In [62]:
# transform the train and test set, and add on the Id and SalePrice variables
data = pd.concat([dataset[['Id', 'SalePrice']].reset_index(drop=True),
                    pd.DataFrame(scaler.transform(dataset[feature_scale]), columns=feature_scale)],
                    axis=1)

### Copilot Prompt::-- ok since the salesprice is a continuous variable and also the target variable and it was skewed I applied log transform on it. Now the variable is not skewed.

### But my question how does this affect the model's prediction? 

### Should I use the skewed salePrice target variable or the normalized salePrice target variable for model training and testing. If I use the unskewed how do predict the real value of salePrice variable For example before log the value was 208500 and after was 12.2476 so the model should output 208500 and not 12.2476. And If I use the skewed one what would it affect the models training and prediction.

## Check word file for copilots response.

In [63]:
data.head()

,Id,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontage_NaN,MasVnrArea_NaN,GarageYrBlt_NaN
0,1,12.247694,0.235294,1.0,0.496892,0.431124,1.0,1.0,1.000000,1.0,1.0,1.00,1.0,0.954545,1.0,1.0,1.0,0.8,0.666667,0.500,0.036765,0.098361,1.0,1.0,1.0,1.0,0.666667,0.716038,0.666667,1.0,1.00,0.75,1.00,1.00,0.833333,0.759508,1.0,0.0,0.646838,0.774668,1.0,1.0,1.0,1.0,0.400467,0.884420,0.0,0.624795,0.333333,0.0,0.666667,0.5,0.375,0.333333,0.666667,0.500000,1.0,0.000000,1.0,1.0,0.046729,0.666667,0.50,0.869159,1.0,1.0,1.0,0.000000,0.654449,0.000000,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.090909,0.50,1.0,1.0,0.0,0.0,0.6
1,2,12.109011,0.000000,1.0,0.573315,0.457169,1.0,1.0,1.000000,1.0,1.0,0.25,1.0,0.136364,0.8,1.0,1.0,1.0,0.555556,0.875,0.227941,0.524590,1.0,1.0,0.8,0.9,1.000000,0.000000,1.000000,1.0,0.75,0.75,1.00,0.50,0.666667,0.797188,1.0,0.0,0.728731,0.819153,1.0,1.0,1.0,1.0,0.549599,0.000000,0.0,0.519684,0.000000,0.5,0.666667,0.0,0.375,0.333333,1.000000,0.333333,1.0,0.333333,0.6,1.0,0.289720,0.666667,0.50,0.845088,1.0,1.0,1.0,0.843935,0.000000,0.000000,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.363636,0.25,1.0,1.0,0.0,0.0,0.4
2,3,12.317167,0.235294,1.0,0.513817,0.489044,1.0,1.0,0.666667,1.0,1.0,1.00,1.0,0.954545,1.0,1.0,1.0,0.8,0.666667,0.500,0.051471,0.114754,1.0,1.0,1.0,1.0,0.666667,0.690361,0.666667,1.0,1.00,0.75,1.00,0.25,0.833333,0.716357,1.0,0.0,0.783246,0.782930,1.0,1.0,1.0,1.0,0.428805,0.886245,0.0,0.639489,0.333333,0.0,0.666667,0.5,0.375,0.333333,0.666667,0.333333,1.0,0.333333,0.6,1.0,0.065421,0.666667,0.50,0.883450,1.0,1.0,1.0,0.000000,0.596422,0.000000,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.727273,0.50,1.0,1.0,0.0,0.0,0.6
3,4,11.849398,0.294118,1.0,0.466412,0.456111,1.0,1.0,0.666667,1.0,1.0,0.75,1.0,0.500000,1.0,1.0,1.0,0.8,0.666667,0.500,0.669118,0.606557,1.0,1.0,0.7,0.4,1.000000,0.000000,1.000000,1.0,0.50,1.00,0.75,1.00,0.666667,0.622780,1.0,0.0,0.811361,0.760436,1.0,0.5,1.0,1.0,0.445797,0.868471,0.0,0.626179,0.333333,0.0,0.333333,0.0,0.375,0.333333,0.666667,0.416667,1.0,0.333333,0.8,0.8,0.074766,1.000000,0.75,0.890935,1.0,1.0,1.0,0.000000,0.568247,0.888227,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.090909,0.00,1.0,0.5,0.0,0.0,0.2
4,5,12.429216,0.235294,1.0,0.590747,0.535696,1.0,1.0,0.666667,1.0,1.0,0.25,1.0,0.409091,1.0,1.0,1.0,0.8,0.777778,0.500,0.058824,0.147541,1.0,1.0,1.0,1.0,0.666667,0.794318,0.666667,1.0,1.00,0.75,1.00,0.75,0.833333,0.750841,1.0,0.0,0.798858,0.808002,1.0,1.0,1.0,1.0,0.512997,0.911832,0.0,0.708471,0.333333,0.0,0.666667,0.5,0.500,0.333333,0.666667,0.583333,1.0,0.333333,0.6,1.0,0.074766,0.666667,0.75,0.927266,1.0,1.0,1.0,0.779126,0.704481,0.000000,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.000000,0.50,1.0,1.0,0.0,0.0,0.6


In [64]:
data.to_csv('X_train1.csv',index=False)